# TP3

## Setup

### Imports

In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm
import numpy as np 
import matplotlib.pyplot as plt  
import statsmodels.api as sm     

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score,roc_curve, roc_auc_score, RocCurveDisplay
from sklearn.preprocessing import StandardScaler

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

### Directories

In [2]:
input_dir = 'input/'
output_dir = 'output/'
EPHs_dir = input_dir + 'EPH_usu_1er_Trim_2023_xlsx/'

## Parte 1
A continuación, complementamos el trabajo hecho en el TP2 usando [la encuesta a
nivel hogares de la EPH](../input/EPH_usu_1er_Trim_2023_xlsx).

1. Exploren el diseño de registro de la base de hogar: a priori, ¿qué variables creen
que pueden ser muy predictivas de pobreza y que sería muy útil incluir para
perfeccionar el ejercicio del TP2?

2. Descarguen la base de microdatos de la EPH correspondiente al primer trimestre
de 2023 (la base de hogares se llama [usu_hogar_T123.xls](//input/EPH_usu_1er_Trim_2023_xlsx/usu_hogar_T123.xlsx)). Importen los datos
de la encuesta de hogar y, al igual que en el TP2, conserven sólo las observaciones
que corresponden a los aglomerados de Ciudad Autónoma de Buenos Aires o
del Gran Buenos Aires.

In [3]:
#Determinamos el directorio de la base de microdatos de la EPH
eph_hog_dir = os.path.join(EPHs_dir, 'usu_hogar_T123.xlsx')

#Leemos y copiamos dicha base
dataframe = pd.read_excel(eph_hog_dir)
df_eph_hog = dataframe.copy()

#Nos quedamos solo con las Obs de CABA y GBA (Aglomerados: 32, 33)
df_eph_hog_BsAs = df_eph_hog[df_eph_hog['AGLOMERADO'].isin([32, 33])]

#Testeamos si lo importamos correctamente:
df_eph_hog_BsAs.head(15)

,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,REALIZADA,REGION,MAS_500,AGLOMERADO,PONDERA,IV1,...,GDECCFR,PDECCFR,ADECCFR,PONDIH,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
9,TQRMNORVYHMOTSCDEIJAH00802517,2023,1,1,1,1,S,33,1066,1,...,12.0,NaN,12,0,1,0,98,0,0,0
10,TQRMNOSQRHLLTTCDEIJAH00719390,2023,1,1,1,1,S,33,2270,2,...,6.0,NaN,7,4733,1,0,2,0,0,0
11,TQSMNOSQRHLLTTCDEIJAH00719389,2023,1,1,1,1,S,33,2161,1,...,7.0,NaN,8,2672,1,0,98,0,0,0
35,TQRMNORTUHKOQQCDEIJAH00780489,2023,1,1,1,1,S,33,3097,1,...,8.0,NaN,9,4844,1,0,98,0,0,0
98,TQRMNOUTRHKNQMCDEIJAH00802590,2023,1,1,1,1,S,33,2571,1,...,8.0,NaN,9,3482,1,0,97,0,0,0
99,TQRMNOQQPHJMQUCDEIJAH00796386,2023,1,1,1,1,S,33,1540,1,...,12.0,NaN,12,0,1,0,98,0,0,0
100,TQRMNOSXXHKMQNCDEIJAH00780852,2023,1,1,1,1,S,33,350,1,...,1.0,NaN,1,753,2,0,3,6,0,0
101,TQRMNOPQXHLNQSCDEIJAH00719335,2023,1,1,1,1,S,33,754,1,...,3.0,NaN,4,1324,2,0,98,0,0,0
102,TQRMNORRRHLNQSCDEIJAH00780835,2023,1,1,1,1,S,33,661,1,...,2.0,NaN,2,1296,2,0,98,0,0,0
103,TQRMNOSPWHMMQTCDEIJAH00802591,2023,1,1,1,1,S,33,1522,1,...,12.0,NaN,12,0,1,0,98,0,0,0


3. Unan la tabla de la encuesta individual con la de la encuesta de hogar.


In [4]:
#Determinamos el directorio de la base de microdatos de la EPH
eph_indiv_dir = os.path.join(EPHs_dir, 'usu_individual_T123.xlsx')

#Leemos y copiamos dicha base
dataframe = pd.read_excel(eph_indiv_dir)
df_eph_indiv = dataframe.copy()
df_eph_indiv = df_eph_indiv[df_eph_indiv['AGLOMERADO'].isin([32, 33])]

#No hace falta filtrar por BsAs ya que con el match de codigo tmb lo hace
df_eph_BsAs = pd.merge(df_eph_indiv, df_eph_hog, on=['CODUSU','NRO_HOGAR'], how='inner', suffixes=('_H', '_I')) #solo se queda con las rows que en ambas bases tienen el mismo CODUSU, 'right' lo haria con hogar y NaN faltantes

#Testeamos si lo importamos correctamente:
with pd.option_context('display.max_columns', None): #nos permite temporalmente sacar la restriccion al limite de cols
  df_eph_BsAs.head(50) 
  



In [5]:
merged_df = df_eph_BsAs.drop(df_eph_BsAs.filter(like='_I').columns, axis=1)

4. Generen sus propias funciones para limpiar la base de datos o, si deciden utilizar funciones existentes en paquetes como numpy y pandas, mencionen cuáles usarán y de qué paquetes son.

In [6]:
def col_drop_if(drop_if_missings=False, other_criteria=[], df=merged_df):
  #dropear si cualquier valor de la lista esta en la columna
  if other_criteria:
    columns_to_drop = [col for col in df.columns if df[col].isin(other_criteria).any()]
    df = df.drop(columns=columns_to_drop)
  
  #dropear si hay missinings(valores vacios)
  if drop_if_missings:
    df.dropna(axis=1, inplace=True)

  return df

def drop_neg_obs(cols=[], df=merged_df):
  #transformamos la columna a valores numericos por las dudas
  df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
  #(Checko si es neg) => Chequeo si algun valor es negativo => If si = True => Iniverto todo con "~" y me quedo solo convalores positivos
  df = df[~(df[cols] < 0).any(axis=1)]
  return df

def replace_rename_col(col=None, replace_dic={}, new_name=None, df=merged_df):
  df[col] = df[col].replace(replace_dic)
  if new_name:
    df = df.rename(columns={col: new_name})
  return df

def categorical_to_dummy(col=None, dummy_names_dic={}, keep_old_col=False, df=merged_df):
  #le agrega a las keys del diccionario la col+"_" como prefijo --> Nos permite ser mas robusto, por si hay valores que no estan en el diccionario, generandolas con nombre identificable igualmente
  complete_dummy_names_dic = {col+ '_' + key: value for key, value in dummy_names_dic.items()}
  #Genera las dummies a partir de los valores categoricos y renombra las dummies segun el diccionario dado
  new_dummies = pd.get_dummies(df[col], prefix=col, dtype=int).rename(columns=complete_dummy_names_dic)
  
  #si keep_old_col = False => drop col vieja
  if ~keep_old_col:
      df = df.drop(columns=[col])

  #le agrega las nuevas dummies al df
  df = pd.concat([df, new_dummies], axis=1)
  return df

In [7]:
df_eph_BsAs_clean = merged_df.copy()

df_eph_BsAs_clean['CH06']

0       65
1       70
2       36
3       28
4        9
        ..
7614     9
7615    77
7616    45
7617    17
7618    26
Name: CH06, Length: 7619, dtype: int64

5. Limpien la base de datos tomando criterios que hagan sentido, tanto para el tratamiento de valores faltantes, de outliers, como así también decidan qué va- riables categóricas y strings usarán y transfórmenlas de forma que haga sentido para los ejercicios siguientes. Justifiquen sus decisiones. 

justificacion

In [8]:
#guardamos el df original por precausion
df_eph_BsAs_clean = merged_df.copy()

df_eph_BsAs_clean = drop_neg_obs(
  cols=['CH06', 'P21', 'IPCF_H', 'ITF_H', 'PP08D1'], df=df_eph_BsAs_clean)

df_eph_BsAs_clean = col_drop_if(
  drop_if_missings=True,
  other_criteria=[], 
  df=df_eph_BsAs_clean)

df_eph_BsAs_clean = replace_rename_col(
  col='MAS_500_H',
  replace_dic={'S':1,'N':0},
  new_name='MAS_500',
  df=df_eph_BsAs_clean)
  
df_eph_BsAs_clean = categorical_to_dummy(
  col='REGION_H', 
  dummy_names_dic={
    '44':'is_Patagonia',
    '40':'is_Noroeste'}, 
  df=df_eph_BsAs_clean)

with pd.option_context('display.max_columns', None):
  df_eph_BsAs_clean.head(10)

C:\Users\ponyl\AppData\Local\Temp\ipykernel_19232\893562635.py:33: DeprecationWarning: Bitwise inversion '~' on bool is deprecated. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  if ~keep_old_col:


In [9]:
merged_df[['CH06']]

,CH06
0,65
1,70
2,36
3,28
4,9
...,...
7614,9
7615,77
7616,45
7617,17


In [10]:
df_eph_BsAs_clean['CH04']

0       2
1       1
2       1
3       2
4       2
       ..
7613    2
7614    1
7615    2
7616    2
7617    1
Name: CH04, Length: 6388, dtype: int64

In [11]:
df_eph_BsAs_clean.isnull().sum()


CODUSU         0
ANO4_H         0
TRIMESTRE_H    0
NRO_HOGAR      0
COMPONENTE     0
              ..
VII2_1         0
VII2_2         0
VII2_3         0
VII2_4         0
REGION_H_1     0
Length: 133, dtype: int64

6. Presenten estadísticas descriptivas de cinco variables de la encuesta de hogar que ustedes creen que pueden ser relevantes para predecir pobreza. 

7. Repitan el inciso 1.2.f del TP2 para construir la columna adulto equiv y la columna ad equiv hogar (pueden utilizar su código del TP2). 

In [12]:
# Definir la tabla de equivalencias necesarias eneréticas según edad y sexo
equiv_energ = { 
  1 : { #si es 1 = Varón
    (0, 0.99): 0.35, #0.35 es el valor de un varón menor a 1 año
    (1, 1): 0.37,
    (2, 2): 0.46,
    (3, 3): 0.51,
    (4, 4): 0.55,
    (5, 5): 0.6,
    (6, 6): 0.64,
    (7, 7): 0.66,
    (8, 8): 0.68,
    (9, 9): 0.69,
    (10, 10): 0.79,
    (11, 11): 0.82,
    (12, 12): 0.85,
    (13, 13): 0.9,
    (14, 14): 0.96,
    (15, 15): 1,
    (16, 16): 1.03,
    (17, 17): 1.04,
    (18, 29): 1.02,
    (30, 45): 1,
    (46, 60): 1,
    (61, 75): 0.83,
    (75, 200): 0.74
  },
  2 : { #si es 2 = Mujer
    (0, 0.99): 0.35,
    (1, 1): 0.37,
    (2, 2): 0.46,
    (3, 3): 0.51,
    (4, 4): 0.55,
    (5, 5): 0.6,
    (6, 6): 0.64,
    (7, 7): 0.66,
    (8, 8): 0.68,
    (9, 9): 0.69,
    (10, 10): 0.7,
    (11, 11): 0.72,
    (12, 12): 0.74,
    (13, 13): 0.76,
    (14, 14): 0.76,
    (15, 15): 0.77,
    (16, 16): 0.77,
    (17, 17): 0.77,
    (18, 29): 0.76,
    (30, 45): 0.77,
    (46, 60): 0.76,
    (61, 75): 0.67,
    (75, 200): 0.63
    }
}

#Función para calcular 'adulto_equiv' basado en la edad y el sexo
def calculate_adulto_equiv(row):
  age = row['Edad'] #Obterner valor de "edad" del la row del df
  sex = row['CH04'] #Obterner valor de "edad" del la row del df
  
  if sex in equiv_energ: #Verificar,por las dudas, si el sexo está en el dic
     # Loopear los rangos de edad y valores en el dic
    for (start, end), value in equiv_energ[sex].items():
        if start <= age <= end: 
            return value #Devolver el valor correcto, si la edad está en el rango
  else: 
    print(f'Not found [age:{age}, sex:{sex}]')
  
  return 0 #devolver 0 como valor predeterminado

df_eph_BsAs_clean.rename(columns={'CH06': 'Edad'}, inplace=True)

df_eph_BsAs_clean['adulto_equiv'] = df_eph_BsAs_clean.apply( #guarda el resultado de la funcion
    lambda row: calculate_adulto_equiv(row), axis=1)#Aplica calculate_adulto_equiv a cada fila del df

#Obtener 'ad_equiv_hogar' para cada row del df con groupby y transform
df_eph_BsAs_clean['ad_equiv_hogar'] = df_eph_BsAs_clean.groupby('CODUSU')['adulto_equiv'].transform('sum')
#Por el problema: base2 & floating point -> redondeamos a 2 decimales(estandar en esta base de datos)
df_eph_BsAs_clean['ad_equiv_hogar'] = df_eph_BsAs_clean['ad_equiv_hogar'].round(2)

8. Repitan el inciso 1.3 y 1.4 del TP2 para dividir la base en dos dataframes donde: uno conserve las personas que no reportaron ITF (dataframe llamado 2 respondieron) y otro conserve a las personas que no reportaron ITF (llama- do norespondieron). Además, agreguen a la base respondieron una colum- na llamada ingreso necesario que sea el producto de la canasta básica por ad equiv hogar. 

In [13]:
# 1.3


# Generamos una dummy llamada respondieron, si indicaron un ingreso mayor a 0 se les asigna un 1
df_eph_BsAs_clean['respondieron'] = (df_eph_BsAs_clean['ITF_H'] > 0).astype(int)


# Sumamos para ver cuanta gente respondio
total_respondieron = df_eph_BsAs_clean['respondieron'].sum()

print(total_respondieron)

# Repetimos para los que no respondieron (es decir los que reportaron un ingreso = 0)

df_eph_BsAs_clean['norespondieron'] = (df_eph_BsAs_clean['ITF_H'] == 0).astype(int)

total_norespondieron = df_eph_BsAs_clean['norespondieron'].sum()

print(total_norespondieron)

# Creamos la base 'respondieron'

respondieron = df_eph_BsAs_clean[df_eph_BsAs_clean['ITF_H'] > 0]

# Creamos la base 'norespondieron'

norespondieron = df_eph_BsAs_clean[df_eph_BsAs_clean['ITF_H']==0]

# 1.4

# Creamos una nueva variable llamada ingreso necesario, que contiene la operacion exigida por la consigna

respondieron.loc[:,"ingreso_necesario"] = respondieron["ad_equiv_hogar"] * 57371.05


print(respondieron['ingreso_necesario'])

4181
2207
0        86056.5750
1        86056.5750
2       140559.0725
3       140559.0725
4       140559.0725
           ...     
7611    219157.4110
7612    141132.7830
7613    141132.7830
7614    141132.7830
7615     36143.7615
Name: ingreso_necesario, Length: 4181, dtype: float64


C:\Users\ponyl\AppData\Local\Temp\ipykernel_19232\4082595519.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  respondieron.loc[:,"ingreso_necesario"] = respondieron["ad_equiv_hogar"] * 57371.05


9. Agreguen a la base respondieron una columna llamada pobre, que tome valor 1 si el ITF es menor al ingreso necesario que necesita esa familia y 0 en caso contrario. 

In [14]:
# Creamos una nueva columna llamada pobre, que contiene un 1 si la persona tiene un ingreso menor al necesario

respondieron['pobre'] = (respondieron['ITF_H'] < respondieron['ingreso_necesario']).astype(int)

C:\Users\ponyl\AppData\Local\Temp\ipykernel_19232\3251145518.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  respondieron['pobre'] = (respondieron['ITF_H'] < respondieron['ingreso_necesario']).astype(int)


10. En el TP2 calcularon los individuos bajo la línea de pobreza. Sin embargo, cuando se habla de pobreza el número más utilizado es el de la tasa de hogares bajo la línea de pobreza. Para calcularlo, utilicen una sola observación por hogar y sumen el ponderador PONDIH que permite expandir la muestra de la EPH al total de la población que representa. ¿Cuál es la tasa de hogares bajo la línea de pobreza para el GBA? ¿Se asemeja al que reporta el INDEC en sus informes?

In [15]:
# Contamos la cantidad de pobres
cantidad_pobres = respondieron['pobre'].sum()
proporción_pobres = cantidad_pobres/len(respondieron['pobre'])

print(cantidad_pobres)
print(proporción_pobres)

1566
0.3745515426931356


## Parte 2

In [16]:

#from ISLP import load_data
import os  
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt  
import statsmodels.api as sm     

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.preprocessing import StandardScaler



from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

1. Escriban una función, llamada evalua metodo, que reciba como argumentos un modelo y los datos de entrenamiento y prueba (X train, y train, X test, y test). La función debe ajustar el modelo con los datos de entrenamiento y calcular las métricas que considere necesarias para esta problemática (de mínima, deben reportar la matriz de confusión, las curvas ROC y los valores de AUC y de accuracy score de cada método). El output de la función debe ser una colección con las métricas evaluadas.

In [30]:
def evalua_metodo(modelo, X_train, X_test, Y_train, Y_test):
    modelo.fit(X_train, Y_train)
    y_pred_modelo = modelo.predict(X_test)[:,1]
    matriz_confusion_modelo = confusion_matrix(Y_test, y_pred_modelo)
    tn, fp , fn, tp = confusion_matrix(Y_test, y_pred_modelo).ravel()
    specificity_modelo = tn / (tn+fn)
    accuracy_modelo = accuracy_score(Y_test, y_pred_modelo)
    recall_modelo = recall_score(Y_test, y_pred_modelo)
    precision_modelo = tp/(tp + fp)
    auc_modelo = roc_auc_score(Y_test, y_pred_modelo)
    fpr, tpr, thresholds = roc_curve(Y_test, y_pred_modelo)
    np.set_printoptions(suppress = True)
    display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc_lda, estimator_name='Análisis discriminante lineal')
    display.plot()  
    plt.plot([0, 1], [0, 1], color='red', linestyle='--')
    plt.show() 
    return matriz_confusion_modelo, specificity_modelo, accuracy_modelo, recall_modelo, presicion_modelo, auc_modelo, 
    
#ver output colección de métricas

In [52]:
def evalua_metodo(modelo, X_train, X_test, Y_test, Y_train, delta, k_knn):
    
    if modelo == LogisticRegression:
        y = LogisticRegression(C=1/delta).fit(X_train, Y_train)
    elif modelo == LinearDiscriminantAnalysis:
        y = LinearDiscriminantAnalysis().fit(X_train, Y_train)
    elif modelo == KNeighborsClassifier:
        y = KNeighborsClassifier(k_knn).fit(X_train, Y_train)

    Y_pred = y.predict(X_test)
    
    matriz_confusion = confusion_matrix(Y_test, Y_pred)
    curva_ROC = roc_auc_score(Y_test, Y_pred)
    valores_AUC = accuracy_score(Y_test, Y_pred)
    
    return matriz_confusion,curva_ROC,valores_AUC

2. Escriban una función, llamada cross validation, que realice validación cru-
zada con k iteraciones (k -fold CV), llamando a la función del inciso anterior en
cada una, pero para las k distintas particiones. La función debe recibir como
argumentos el modelo, el valor de k y un dataset (es decir, sólo X e y). Pueden
ayudarse con la función KFold para generar las particiones necesarias.


In [19]:
def cross_validation(modelo, K, X, Y):
    kf = KFold(n_splits=K, shuffle=True, random_state=100)
    for i, (train_index, test_index) in enumerate(kf.split(X)):   
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        modelo = evalua_metodo(modelo, X_train, X_test, Y_train, Y_test)[1]    
        ecms = ecms.append({"modelo": modelo, "particion": i, "ecm": ecm}, ignore_index=True)    

3. Escriban una función, llamada evalua config que reciba una lista de configuraciones de hiperparámetros1 (los distintos valores a probar como hiper
parámetros podrian codificarse en diccionarios de Python) y utilizando la función cross validation obtenga el error2 promedio para cada configuración.
En scikit-learn, muchos métodos llaman penalty al método de regularización y C a la inversa
del hiperparámetro λ.
Utilicen la medición del error que prefieran. Una opción ser ́ıa el Error Cuadrático Medio.
Finalmente, la función debe devolver la configuración que genere menor error3.


In [20]:
# hiperparametros = #crear lista de landas posibles

def evalua_config (hiperparametros, K, X, Y):
    ret={}
    for i in hiperparametros:
        reg =  LogisticRegression(C=1/i, penalty="elasticnet")
        ecm=cross_validation(reg,k, X, Y)
        ret[elem]= ecm


    return min(ret, key=ret.get)

4. Escriban una función llamada evalua multiples metodos que les permita im-
plementar los siguiente métodos con los hiperparámetros que ustedes elijan.
Para la regresión log ́ıstica, asegúrense de que esta función utilice su función
evalua config para optimizar el λ de la regularización. Finalmente, el output
de la función debe ser una tabla donde las columnas sean las métricas que hayan
evaluado (las que hayan incluido en la función evalua metodo) y las filas sean
los modelos (con su configuración de hiperparámetros asociada) que hayan co-
rrido. Asegúrense de que la tabla incluya una columna con nombre del modelo
y el valor de los hiperparámetros/configuración:4
Regresión log ́ıstica
Análisis de discriminante lineal
KNN

In [36]:
k_knn = 3 #usamos la misma cantidad de vecinos cercanos que la otra vez

def evalua_multiples_metodos (K, X_train, X_test , Y_train, Y_test, X, Y, hiperparametro_optimo, k_knn):

    modelos = [LogisticRegression, KNeighborsClassifier, LinearDiscriminantAnalysis]

    #landas= np.arange(0.01, landa_max,10).tolist() --> crear bien lista de landas posibles

    matriz = pd.DataFrame(columns=["Modelo", "Hiperparametro","Precisión", "AUC", "ECM"])
    for modelo in modelos:
        if modelo == LogisticRegression:
            hiperparametro_optimo = evalua_config(LogisticRegression, K, X, Y) #hiperparametro landa
            metricas_log= evalua_metodo(modelo, X_train, X_test, Y_train, Y_test, hiperparametro_optimo, k_knn)
            results = [modelo, hiperparametro_optimo, metricas_log['accuracy'][0], metricas_log['auc'][0], metricas_log['ecm'][0]]
            matriz.loc[len(matriz)] = results

        elif modelo == KNeighborsClassifier:
            metricas_kvc = evalua_metodo(modelo, X_train, X_test, Y_train, Y_test, delta, k_knn)
            results = [modelo, k_knn, metricas_kvc['accuracy'][0], metricas_kvc['auc'][0], metricas_kvc['ecm'][0]]
            matriz.loc[len(matriz)] = results

        elif modelo == LinearDiscriminantAnalysis:
            metricas_ad = evalua_metodo(modelo, X_train, X_test, Y_train, Y_test, delta, k_knn)
            results = [modelo, "NA", metricas_ad['accuracy'][0], metricas_ad['auc'][0], metricas_ad['ecm'][0]]
            matriz.loc[len(matriz)] = results


    return(matriz)

## Parte 3

1. Eliminen de ambas bases (respondieron, norespondieron) todas las variables
relacionadas a ingresos (en el archivo Dise ̃no de bases y estructura ver las ca-
tegor ́ıas: ingresos de la ocupaci ́on principal de los asalariados, ingresos de la
ocupaci ́on principal, ingresos de otras ocupaciones, ingreso total individual, in-
gresos no laborales, ingreso total familiar, ingreso per c ́apita familiar). Eliminen
tambi ́en las columnas adulto equiv, ad equiv hogar e ingreso necesario.
Establezcan a la variable pobre como su variable dependiente (vector y). El
resto de las variables ser ́an las variables independientes (matriz X). Dependien-
do de la funci ́on que usen, no se olviden de agregar la columna de 1 cuando sea
necesario.


In [22]:
columnas_a_eliminar = ['PP06A', 'PP06C', 'PP06D', 'PP06E', 'PP06H', 'PP08D1', 'PP08D4', 'PP08F1', 'PP08F2', 'PP08J1', 'PP08J2', 
                       'PP08J3', 'P21', 'DECOCUR', 'IDECOCUR', 'RDECOCUR', 'GDECOCOCUR', 'PDECOCUR', 'ADECOCUR', 'PONDIIO', 
                       'TOT_P12', 'P47T', 'DECINDR', 'IDECINDR', 'RDECINDR', 'GDECINDR', 'PDECINDR', 'ADECINDR', 'ADECINDR', 
                       'PONDII', 'V2_M', 'V3_M', 'V4_M', 'V5_M', 'V8_M', 'V9_M', 'V10_M', 'V11_M', 'V12_M', 'T_VI', 'ITF',
                       'DECIFR', 'IDECIFR', 'RDECIFR', 'GDECIFR', 'PDECIFR', 'ADECIFR', 'IPCF', 'DECCFR', 'IDECCFR', 'RDECCFR',
                       'GDECCFR', 'PDECCFR', 'ADECCFR', 'PONDIH', 'adulto_equiv','ad_equiv_hogar','ingreso_necesario', 'CODUSU', 'CH05']

#Reescribimos los dataframes con las variables que nos interesan
columnas_a_mantener = [col for col in respondieron.columns if col not in columnas_a_eliminar]

columnas_a_mantener2 = [col for col in norespondieron.columns if col not in columnas_a_eliminar]


respondieron = respondieron[columnas_a_mantener]
norespondieron = norespondieron[columnas_a_mantener2]

string_columns = respondieron.select_dtypes(include=['object'])

# Mostrar las columnas con datos de tipo string
if not string_columns.empty:
    print("Las siguientes columnas contienen datos de tipo string:")
    print(string_columns.columns)
else:
    print("No hay columnas con datos de tipo string en el DataFrame.")

No hay columnas con datos de tipo string en el DataFrame.


2. Corran la funci ́on evalua multiples metodos con la base respondieron. En
los pr ́oximos incisos profundizaremos en la tarea de regularizaci ́on, pero en este
ejercicio prueben al menos un hiperpar ́ametro para regularizar y al menos un
valor de λ.
3Consejo: cuanto m ́as gen ́erica construyan la funci ́on, luego podr ́a ser utilizada en m ́as situacio-
nes. Por ahora, la usaremos solo para buscar el λ  ́optimo cuando utilicemos regularizaci ́on.
4Pista: para la regresi ́on log ́ıstica, cuando incluyan regularizaci ́on observen que deber ́an correr
la funci ́on evalua metodo dos veces. Una para optimizar los hiperpar ́ametros (con un set de datos
para train y otro para validaci ́on) y otra para obtener las m ́etricas con el hiperpar ́ametro  ́optimo
(con un set de datos para train y otro para test).

In [47]:
Y = respondieron['pobre']
X_int = respondieron.copy()

Y

0       0
1       0
2       1
3       1
4       1
       ..
7611    1
7612    0
7613    0
7614    0
7615    0
Name: pobre, Length: 4181, dtype: int32

In [54]:
# Eliminamos pobre de respondieron para que no sea la variable a predecir y predictora a la vez. 
if 'pobre' in X_int.columns:
    X_int = X_int.drop('pobre', axis=1) 
    
#Agregamos la columna de unos.
X = sm.add_constant(X_int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state= 211)

print(evalua_multiples_metodos(5, X_train, X_test , Y_train, Y_test, X, Y, 10, 3))


ValueError: Found input variables with inconsistent numbers of samples: [2926, 1255]

In [49]:
X_int

,ANO4_H,TRIMESTRE_H,NRO_HOGAR,COMPONENTE,H15,MAS_500,AGLOMERADO_H,PONDERA_H,CH03,CH04,...,IX_MAYEQ10,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4,REGION_H_1,respondieron,norespondieron
0,2023,1,1,1,1,1,33,1545,1,2,...,2,1,0,2,0,0,0,1,1,0
1,2023,1,1,2,1,1,33,1545,2,1,...,2,1,0,2,0,0,0,1,1,0
2,2023,1,1,1,1,1,33,8423,1,1,...,2,2,1,98,0,0,0,1,1,0
3,2023,1,1,2,1,1,33,8423,2,2,...,2,2,1,98,0,0,0,1,1,0
4,2023,1,1,3,0,1,33,8423,3,2,...,2,2,1,98,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7611,2023,1,1,5,0,1,33,3232,3,1,...,3,1,0,3,4,0,0,1,1,0
7612,2023,1,1,1,1,1,33,1236,1,1,...,2,1,2,98,0,0,0,1,1,0
7613,2023,1,1,2,1,1,33,1236,2,2,...,2,1,2,98,0,0,0,1,1,0
7614,2023,1,1,3,0,1,33,1236,3,1,...,2,1,2,98,0,0,0,1,1,0


3. Expliquen c ́omo elegir ́ıan λ por validaci ́on cruzada. Detallen por qu ́e no usar ́ıan
el conjunto de prueba (test) para su elecci ́on.


4. En validaci ́on cruzada, ¿cu ́al es el problema de usar un k muy peque ̃no y uno
muy grande? Cuando k = n (con n el n ́umero de muestras), ¿cu ́antas veces se
estima el modelo?


5. Realicen un barrido en λ = 10n con n ∈ {−5, −4, −3 . . . , +4, +5} y utilicen 10
fold CV para elegir el λ  ́optimo en regresi ́on logistica con Ridge y con LASSO.
¿Qu ́e λ seleccion ́o en cada caso? Generen box-plots mostrando la distribuci ́on
del error de predicci ́on para cada λ. Cada box debe corresponder a un valor de λ
y contener como observaciones el error medio de validaci ́on para cada partici ́on.
Adem ́as, para la regularizaci ́on LASSO, genere un box-plot similar, pero ahora
graficando la proporci ́on de variables ignoradas por el modelo en funci ́on de λ,
es decir la proporci ́on de variables para las cuales el coeficiente asociado es cero.


6. En el caso del valor  ́optimo de λ para LASSO encontrado en el inciso anterior,
¿qu ́e variables fueron descartadas? ¿Son las que hubieran esperado? ¿Tiene re-
laci ́on con lo que respondieron en el inciso 1 de la Parte I?


7. Elijan alguno de los modelos de regresi ́on log ́ıstica donde hayan probado distin-
tos parametros de regularizaci ́on y comenten: ¿Qu ́e m ́etodo de regularizaci ́on
funcion ́o mejor Ridge o LASSO? Comenten mencionando el error cuadr ́atico
medio (ECM).


8. ¿Cu ́al de todos los m ́etodos evaluados predice mejor? ¿Con qu ́e hiperpar ́ame-
tros? Justifiquen detalladamente utilizando las medidas de precisi ́on que cono-
cen.


9. Con el m ́etodo que seleccionaron, predigan qu ́e personas son pobres dentro de
la base norespondieron. ¿Qu ́e proporci ́on de los hogares son pobres en esa
submuestra?